In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [2]:
spark = SparkSession.builder.master('local[*]').appName('Question 1 array').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/24 12:56:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
rating_df = spark.read.options(sep=';', header=True, inferSchema=True).csv('/home/maryam/content/BooksRating-CSV/Book-Ratings.csv')

In [4]:
rating_df.columns

['userid', 'isbn', 'rate']

In [5]:
rating_df_userid = rating_df.filter(rating_df['userid'] == 276762)

In [14]:
rating_df_userid.show(25)

+------+-----------+----+
|userid|       isbn|rate|
+------+-----------+----+
|276762| 034544003X|   0|
|276762| 0380000059|   0|
|276762| 0380711524|   5|
|276762| 0451167317|   0|
|276762| 0451454952|   0|
|276762| 0843920262|   0|
|276762| 3404122879|   0|
|276762| 3404182928|   0|
|276762| 3404611306|   0|
|276762|  342662429|   0|
|276762| 3426690179|   0|
|276762| 3442424216|   0|
|276762| 3442425573|   0|
|276762| 3453092007|   8|
|276762| 3453157745|   0|
|276762| 3453176944|   0|
|276762| 3453185137|   0|
|276762| 3453185323|   0|
|276762| 3453213025|   3|
|276762| 3453877241|   0|
|276762| 3492226604|   0|
|276762| 3517017442|   0|
|276762| 3596125006|   0|
|276762| B0000BLD7X|   0|
|276762|N3453124715|   4|
+------+-----------+----+



In [9]:
# Group by 'userid' and collect 'rate' values into an array
grouped_df = rating_df_userid.groupBy("userid").agg(collect_list("rate").alias("rates"))

In [12]:
grouped_df.show(truncate=False)

+------+---------------------------------------------------------------------------+
|userid|rates                                                                      |
+------+---------------------------------------------------------------------------+
|276762|[0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 4]|
+------+---------------------------------------------------------------------------+



In [22]:
# Join back to the original DataFrame to ensure each row has the array 
final_df = rating_df_userid.join(grouped_df, on="userid")

In [25]:
final_df.show(25, truncate=False)

+------+-----------+----+---------------------------------------------------------------------------+
|userid|isbn       |rate|rates                                                                      |
+------+-----------+----+---------------------------------------------------------------------------+
|276762|034544003X |0   |[0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 4]|
|276762|0380000059 |0   |[0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 4]|
|276762|0380711524 |5   |[0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 4]|
|276762|0451167317 |0   |[0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 4]|
|276762|0451454952 |0   |[0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 4]|
|276762|0843920262 |0   |[0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 4]|
|276762|3404122879 |0   |[0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 3,

# question 2

In [29]:
# Group by 'userid' and collect 'rate' values into an array
grouped_df = rating_df_userid.groupBy("userid").agg(collect_list("rate").alias("rates"))

In [30]:
grouped_df.show(truncate=False)

+------+---------------------------------------------------------------------------+
|userid|rates                                                                      |
+------+---------------------------------------------------------------------------+
|276762|[0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 4]|
+------+---------------------------------------------------------------------------+



In [33]:
# Example 2: Explode array into multiple rows 
exploded_df = grouped_df.withColumn("rate", explode(col("rates"))).drop('rates') 

In [35]:
exploded_df.show(25)

+------+----+
|userid|rate|
+------+----+
|276762|   0|
|276762|   0|
|276762|   5|
|276762|   0|
|276762|   0|
|276762|   0|
|276762|   0|
|276762|   0|
|276762|   0|
|276762|   0|
|276762|   0|
|276762|   0|
|276762|   0|
|276762|   8|
|276762|   0|
|276762|   0|
|276762|   0|
|276762|   0|
|276762|   3|
|276762|   0|
|276762|   0|
|276762|   0|
|276762|   0|
|276762|   0|
|276762|   4|
+------+----+

